In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle= False)
# param_grid = {'alpha': [0.1, 1, 10, 100]}
# ridge = Ridge()
# grid_search = GridSearchCV(ridge, param_grid)
# grid_search.fit(X_train, y_train)
# best_alpha = grid_search.best_params_['alpha']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle= False)
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 30],
#     'min_samples_split': [2, 5, 10]
# }
# grid_search = GridSearchCV(RandomForestRegressor(), param_grid)
# grid_search.fit(X_train, y_train)
# best_alpha = grid_search.best_params_

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle= False)
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'max_depth': [3,4,5,6,7,8],
    'n_estimators': [50, 100, 200, 300, 400]
}
grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_

In [ ]:
def regresion_con_validacion_cruzada_y_test(X, y):
  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle= False)
    
    # Inicializa los modelos
    modelos = {
        "Ridge": Ridge(alpha=100),
        "Lasso": Lasso(alpha=100),
        "XGB": XGBRegressor(learning_rate= 0.01, max_depth= 4, n_estimators=300),
        "Random Forest": RandomForestRegressor(n_estimators=300, max_depth=30 ,min_samples_split=10 ,random_state=42)
    }
    
 
    resultados = {}
    for nombre, modelo in modelos.items():
        mse_scores = []
        r2_scores = []
        mae_scores = []
        mape_scores = []
        
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        for train_index, val_index in kf.split(X_train):
            
            train_index = train_index.astype('int64')
            X_train_fold = X_train.iloc[train_index]
            X_val_fold = X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
            
            modelo.fit(X_train_fold, y_train_fold)
            y_pred = modelo.predict(X_val_fold)
            
            mse_fold = mean_squared_error(y_val_fold, y_pred)
            r2_fold = r2_score(y_val_fold, y_pred)
            mae_fold = mean_absolute_error(y_val_fold, y_pred)
            mape_fold = (mean_absolute_percentage_error(y_val_fold, y_pred)*100)
            
            mse_scores.append(mse_fold)
            r2_scores.append(r2_fold)
            mae_scores.append(mae_fold)
            mape_scores.append(mape_fold)
        

        mse_promedio = np.mean(mse_scores)
        r2_promedio = np.mean(r2_scores)
        mae_promedio = np.mean(mae_scores)
        mape_promedio = np.mean(mape_scores)
        
        # Evalúa el modelo en el conjunto de prueba
        modelo.fit(X_train, y_train)
        y_pred_test = modelo.predict(X_test)
        mse_test = mean_squared_error(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)
        mae_test =  mean_absolute_error(y_test, y_pred_test)
        mape_test =  (mean_absolute_percentage_error(y_test, y_pred_test)*100)
        
        resultados[nombre] = {
            'MSE Promedio': mse_promedio,
            'R² Promedio': r2_promedio,
            'MAE Promedio': mae_promedio,
            'MAPE Promedio': mape_promedio,
            'MSE en Test': mse_test,
            'R² en Test': r2_test,
            'MAE en Test': mae_test,
            'MAPE en Test': mape_test
            
        }
    
    return resultados


resultados = regresion_con_validacion_cruzada_y_test(X, y)


for modelo, metricas in resultados.items():
    print(f"Modelo: {modelo}")
    print(f"MSE Promedio en Validación Cruzada: {metricas['MSE Promedio']}")
    print(f"R² Promedio en Validación Cruzada: {metricas['R² Promedio']}")
    print(f"MAE Promedio en Validación Cruzada: {metricas['MAE Promedio']}")
    print(f"MAPE Promedio en Validación Cruzada: {metricas['MAPE Promedio']}")    
    print(f"MSE en Test: {metricas['MSE en Test']}")
    print(f"R² en Test: {metricas['R² en Test']}")
    print(f"MAE en Test: {metricas['MAE en Test']}")
    print(f"MAPE en Test: {metricas['MAPE en Test']}")
    print()